In [6]:
import pickle
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.datasets import make_moons
from scipy.linalg import fractional_matrix_power

def load_data(file):
    data_list = []
    label_list = []
    for i in range(1,6):
        with open(f"{file}/data_batch_{i}", 'rb') as f:
            _data = pickle.load(f, encoding="bytes")
            #import pdb; pdb.set_trace()
            data_list.append(_data[b'data'])
            label_list.append(_data[b'labels'])
    return (data_list, label_list)
            
data_cifar10 = load_data("./cifar10")

In [7]:
data, label = data_cifar10
data = np.concatenate(data)
label = np.concatenate(label)

In [17]:

alpha = 0.99
sigma = 0.1
num_iter = 1000
num_label_per_cls = 10



In [13]:




def generate_affinity_matrix(X, Sigma=0.01,metric="L2"):
    W = np.sum((X[np.newaxis,:,:] - X[:,np.newaxis,:])**2, axis=-1)
    #import pdb;pdb.set_trace()
    W = np.exp(- W / (2*Sigma**2))
    np.fill_diagonal(W, 0)
    D_n12 = np.sum(W, axis=1)**(-1/2)
    D_n1 = np.sum(W, axis=1)**(-1)
    D_n12 = np.diag(D_n12)
    D_n1 = np.diag(D_n1)
    return W, D_n12, D_n1

def walk_matrix(X, Sigma=0.01, m_type='norm'):
    if m_type == 'norm':
        W, D_n12, _ = generate_affinity_matrix(X, Sigma=Sigma)
        S = D_n12 @ W @ D_n12
        return S
    elif m_type == 'random':
        W, _, D_n1 = generate_affinity_matrix(X, Sigma=Sigma)
        S = W @ D_n1
        return  S
    else:
        raise NotImplemented()

def label_matrix(label_vec):
    #one-hot
    n_values = np.max(label_vec) + 1
    one_hot_y = np.eye(n_values)[label_vec]
    return one_hot_y

        
def lgc(S, Y, alpha=0.99,num_I=400):
    
    # Iteration 0/ F_0 = Y
    F = Y
    for t in range(num_I):
        F = S @ F * alpha + (1-alpha)*Y
    
    return F



In [24]:
labeled_num = 500
sample_num = 1000
Y = np.zeros([sample_num, 10])
data_exp = (data[:sample_num] - np.mean(data[:sample_num]))/(np.var(data[:sample_num]))
target = label[:sample_num]
matrix = label_matrix(label[:labeled_num])
Y[:labeled_num] = matrix
S = walk_matrix(data_exp, Sigma=sigma, m_type='norm')
F = lgc(S, Y, alpha, num_iter)

pred_y = np.argmax(F, axis=-1)
np.mean(pred_y[labeled_num:] == target[labeled_num:])

> /tmp/ipykernel_6160/3803365494.py(13)generate_affinity_matrix()
     11     W = np.sum((X[np.newaxis,:,:] - X[:,np.newaxis,:])**2, axis=-1)
     12     import pdb;pdb.set_trace()
---> 13     W = np.exp(- W / (2*Sigma**2))
     14     np.fill_diagonal(W, 0)
     15     D_n12 = np.sum(W, axis=1)**(-1/2)

ipdb> c


0.222

In [16]:
pred_y = np.argmax(F, axis=-1)
np.mean(pred_y[200:] == target[200:])

0.26